# WORK IN PROGRESS

### Implementations

Supervised
- kNN
- Decision Trees
    - Gini 
    - Ent
- Naive Bayse
- Regression
    - Simple Linear
    - Multiple
    - Logigistic
- Perceptrons
- FeedForawd Neural Nets

Unsupervised:
- k-means

### Imports we are going to use

In [1]:
# Numpy for Array operations
import numpy

#Pandas for nice data layout also will make it easer to implement the stats at the end
import pandas

from collections import Counter #for counting :D 
import math # ~~maths~~ 
import random # for when we are feeling random 0-o

#This is literally just to ensure our sublasses will be required to implement abstract methods
from abc import ABCMeta, abstractmethod

# Main Class

So I have gone with a fairly boring name for our base class. its just ML because why not. 
This base class will wrap up some functionality that each of our methods should have.
- Ensuring Similarity [Done]
- Train test splitting [Done]
- testing [Done]
- Evaluation []

In [2]:
class MachineLearning(metaclass=ABCMeta):
    
    def __init__(self, x, y, tts, thresh):
        x_ = x.copy()
        y_ = y
        if type(x) == pandas.DataFrame:
            if 'labels' in x_.columns:
                y_ = x_.pop('labels')
            else:
                if y_ is not None:
                    raise RuntimeWarning("No 'labels' provided in Dataframe. Using the passed y")
            x_ = x_.values
        if type(x_) != numpy.ndarray:
            x_ = numpy.array(x_)
        if y_ is not None:
            if type(y_) != numpy.ndarray:
                y_ = numpy.array(y_)
            if (len(x_) != len(y_)):
                raise ValueError("X and Y should have the same amount of samples")
            if len(y_.shape) > 1:
                raise ValueError("Y values should be singular")
        self.xtrain, self.ytrain, self.xtest, self.ytest = self.__tts__(x_, y_, tts, thresh)
        self.num_samples = len(self.xtrain) - 1
        self.num_features = len(self.xtrain[0]) - 1
            
        
    def get_train(self):
        #Get the Training Data in a nice way
        zipped_ = zip([i for i in range(len(self.xtrain))], self.xtrain.transpose())
        df_data = {}
        for n, sample in zipped_:
            df_data["train_feature"+str(n)] = sample
        if self.ytrain is not None:
            df_data["train_labels"] = self.ytrain
        return pandas.DataFrame(df_data)
            
    def get_test(self):
        testd = {}
        if self.xtest is None:
            raise RuntimeWarning("No Test Data")
        else:
            zipped_ = zip([i for i in range(len(self.xtest))], self.xtest.transpose())
            df_data = {}
            for n, sample in zipped_:
                testd["test_feature_"+str(n)] = sample
            if self.ytest is not None:
                testd["test_labels"] = self.ytest
        return pandas.DataFrame(testd)
    
    def __tts__(self, x, y, tts = 0.8, thresh = 10):
        '''
        This function just splits the data
        the threshold just checks whether or not to split the data
        we don't need this but sometimes if you're just experimenting
        you might have small datasets and not want to split it and you will want to Test
        on the Training (Even if it screws the evaluation)
        '''
        xtrain, ytrain, xtest, ytest = None, None, None, None
        xlen = len(x)
        train_len = round(xlen * tts)
        test_len = xlen - train_len
        if y is not None:
            if thresh > test_len:
                xtrain, xtest = x.copy(), x.copy()
                ytrain, ytest = y.copy(), y.copy()
            else:
                xtrain, xtest = x.copy()[0:train_len], x.copy()[train_len:]
                ytrain, ytest = y.copy()[0:train_len], y.copy()[train_len:]
        else:
            xtrain, xtest = x.copy(), x.copy()
        return xtrain, ytrain, xtest, ytest
        
    @abstractmethod
    def fit(self):
        pass
    
    @abstractmethod
    def predict(self, x):
        pass
    
    def __can_test__(self):
        if self.ytest is None:
            raise RuntimeError("No test labels")
        return True
    
    def predict_testx(self):
        predictions = self.predict(self.xtest)
        test_res = {"predictions": predictions}
        return test_res
    
    def test(self):
        self.__can_test__()
        test_res = self.predict_testx()
        test_res["Actual"] = self.ytest
        return pandas.DataFrame(test_res)
    
    def binary_results(self):
        self.__can_test__()
        res = self.test()
        tp = len(res[(res["Actual"] == 1) & (res["predictions"] == 1)])
        fn = len(res[(res["Actual"] == 1) & (res["predictions"] == 0)])
        fp = len(res[(res["Actual"] == 0) & (res["predictions"] == 1)])
        tn = len(res[(res["Actual"] == 0) & (res["predictions"] == 0)])
        return tp, fn, fp, tn
    
    def binary_stats(self):
        self.__can_test__()
        tp, fn, fp, tn =  self.binary_results()
        tp, fn, fp, tn = tp+1, fn+1, fp+1, tn+1 
        acc = (tp + tn) / (tp + fn + fp + tn)
        prec = tp / (tp + fp)
        recall = tp / (tp + fn)
        speci = tn / (tn + fp)
        # 𝑭𝟏 = 𝟐 ∗ (𝒑𝒓𝒆𝒄𝒊𝒔𝒊𝒐𝒏 ∗ 𝒓𝒆𝒄𝒂𝒍𝒍) / (𝒑𝒓𝒆𝒄𝒊𝒔𝒊𝒐𝒏 + 𝒓𝒆𝒄𝒂𝒍𝒍)
        f1 = 2 * (prec * recall) / (prec + recall)
        return {
            "Acuuracy": round(acc,3),
            "Precision": round(prec,3),
            "Recall": round(recall,3),
            "Specificity": round(speci,3),
            "F1": round(f1,3)
        }
    
    def confusion_matrix(self):
        self.__can_test__()
        tp, fn, fp, tn = self.results()
        d = {
            "pos": [tp, fp],
            "neg": [fn, tn]
        }
        ret = pandas.DataFrame(d)
        ret.rename(index={0: "pos", 1: "neg"})
        return ret

# K-Nearest Neighbours

In [3]:
class kNN(MachineLearning):
    
    def __init__(self, x, y=None, k=3, weighted=False, dist_type="manhattan", tts=0.8, thresh=10):
        super(kNN, self).__init__(x, y, tts=0.8, thresh=10)
        self.k = k
        self.weighted = weighted
        self.dist_type = dist_type
        self.distance_types = ["manhattan", "euclid"]
        if self.dist_type not in self.distance_types:
            raise RuntimeError("You need to use an actual distance type (see kNN.distance_types)")
        if self.k > self.num_samples:
            raise RuntimeError("k can't be greater than the number of samples")
        if self.ytrain is None:
            raise RuntimeError("K-Nearest Neighbours needs labels")
        
    def fit(self, x, y=None):
        #For K-neighbours we don't need to fit since we predict on the samples.
        pass
    
    def distance(self, a, b):
        tot = 0
        for i in range(self.num_features):
            if self.dist_type == "manhattan":
                tot += abs(a[i] - b[i])
            elif self.dist_type == "euclid":
                tot += ((a[i] - b[i]) ** 2)
        if self.dist_type == "euclid":
            tot = math.sqrt(tot)
        return tot
    
    def calc_weighted_label(self, labels, weights):
        tot_pos, tot_neg = 0, 0
        for i in range(self.k):
            if labels[i] == 0:
                tot_neg += weights[i]
            else:
                tot_pos += weights[i]
        ret = 0
        if tot_pos > tot_neg:
            ret = 1
        if tot_pos == tot_neg:
            ret = 1 if (random.random() > 0.5) else 0
        return ret
    
    def calc_knn(self, dists):
        ret = 0
        s = sorted(dists)
        labs, weights = [], []
        for i in range(self.k):
            l = self.ytrain[dists.index(s[i])]
            labs.append(l)
            weights.append(s[i])
        if self.weighted:
            ret = self.calc_weighted_label(labs, weights)
        else:
            c = Counter(labs)
            ret = c.most_common()[0][0]
        return ret
    
    def knn_sample(self, n):
        distances = []
        for i in range(self.num_samples):
            dist = self.distance(self.xtrain[i], n)
            distances.append(dist)
        return self.calc_knn(distances)
    
    def predict(self, x):
        ret = []
        if type(x) != numpy.ndarray:
            if type(x) != list:
                raise RuntimeError("You need to provide the values as a list")
        for samp in x:
            ret.append(self.knn_sample(samp))
        return numpy.array(ret)
        

## KNN EVAL

In [4]:
df = pandas.read_csv("./diabetes.csv")
df = df.rename(columns={"Outcome": "labels"})
labs = df.pop("labels") 
df = (df - df.mean())/df.std(ddof=0)
df["labels"] = labs

In [5]:
knn = kNN(df, k=8)
print(knn.binary_stats())
knn = kNN(df, k=8, weighted=True)
print(knn.binary_stats())
knn = kNN(df, k=8, dist_type="euclid")
print(knn.binary_stats())
knn = kNN(df, k=8, weighted=True, dist_type="euclid")
print(knn.binary_stats())

{'Acuuracy': 0.747, 'Precision': 0.681, 'Recall': 0.561, 'Specificity': 0.851, 'F1': 0.615}
{'Acuuracy': 0.715, 'Precision': 0.63, 'Recall': 0.509, 'Specificity': 0.832, 'F1': 0.563}
{'Acuuracy': 0.722, 'Precision': 0.644, 'Recall': 0.509, 'Specificity': 0.842, 'F1': 0.569}
{'Acuuracy': 0.709, 'Precision': 0.641, 'Recall': 0.439, 'Specificity': 0.861, 'F1': 0.521}


# Decision Trees

In [6]:
class DecisionTree(MachineLearning):
    
    def __init__(self, x, y=None, impurity_measure="gini", max_depth=3, tts=0.8, thresh=10):
        super(DecisionTree, self).__init__(x, y, tts=0.8, thresh=10)
        self.max_depth = max_depth
        self.tree = None
        if self.ytrain is None:
            raise RuntimeError("Decision Tree needs labels")
        self.impurity_measure = impurity_measure
        
        self.__f__ = None
        if impurity_measure == "gini":
            self.__f__  = lambda x: (x * (1 - x))
        elif impurity_measure == "entropy":
            self.__f__  = lambda x: (-1 * x * math.log(x, 2))
        
        self.__rename__  = None
        self.__df__ = pandas.DataFrame(self.xtrain)
        if type(x) == pandas.DataFrame:
            z = zip(list(df.columns.factorize()[0]),
                   list(df.columns.factorize()[1]))
            d = {key: value for (key, value) in z} 
            self.__df__ = self.__df__.rename(columns=d)
            self.__rename__ = d
        self.__cols__ = self.__df__.columns
        self.__df__["labels"] = self.ytrain
        self.__classes__ = numpy.unique(self.__df__["labels"])
        self.fit()
        
        
    def __feature_split__(self, dataset, feature):
        split = dataset[feature].median()
        above = dataset[dataset[feature] >= split]
        below = dataset[dataset[feature] < split]
        return above, below
    
    def __feature_impurity__(self, dataset, feature):
        tot = len(dataset)
        ab, be = self.__feature_split__(dataset, feature)
        ab_imp = ( (len(ab)/tot) * self.__impurity__(Counter(ab["labels"])))
        be_imp = ( (len(be)/tot) * self.__impurity__(Counter(be["labels"])))
        return (ab_imp + be_imp)
        
    def __impurity__(self, counter):
        total = 10000
        tot_e = sum(counter.values())
        if tot_e != 0:
            total = 0
            for e in self.__classes__:
                p_e = (counter[e]/tot_e)
            total += (0 if p_e == 0 else self.__f__(p_e))
        return float(total)
    
    def __best_feature__(self, dataset):
        h_s = self.__impurity__(Counter(dataset["labels"]))
        feature = self.__cols__[0]
        feature_imp = -1
        for f in self.__cols__ :
            imp_f = self.__feature_impurity__(dataset, f)
            f_imp = (h_s - imp_f)
            if f_imp > feature_imp:
                feature = f
                feature_imp = f_imp
        return feature
    
    def __node_info__(self, dataset, d):
        ret = None
        if len(dataset) > 0:
            c = Counter(dataset["labels"])
            impurity = self.__impurity__(c) 
            ret =  {
                "depth": d,
                "classes": dict(c),
                "impurity": impurity,
                "leaf": True
            }
            if (impurity != 0):
                split_feat = self.__best_feature__(dataset)
                bar = dataset[split_feat].median()
                question = split_feat+" >= "+str(bar) 
                ret["leaf"] = False
                ret["split_feature"] = split_feat
                ret["above"] = bar
                ret["question"] = question
        if ret["leaf"]:
            ret["class"] = c.most_common()[0][0]
        return ret
        
    def __create_tree__(self, dataset, depth):
        ds = dataset.copy()
        node = self.__node_info__(ds, depth)
        if (node is not None) and (node["impurity"] != 0):
            a, b = self.__feature_split__(ds, node["split_feature"])
            if depth != self.max_depth:  
                if len(a) > 0:
                    node["True"] = self.__create_tree__(a, (depth + 1))
                if len(b) > 0:
                    node["False"] = self.__create_tree__(b, (depth + 1))
            else:
                node["leaf"] = True
        return node
            
    def fit(self):
        top_node = self.__create_tree__(self.__df__.copy(), 0)
        self.tree = top_node
    
    def predict_sample(self, x):
        curr_q = self.tree
        while (curr_q["impurity"] != 0.0):
            q = curr_q["split_feature"]
            a = curr_q["above"]
            curr_q = curr_q[str((x[q] >= a).item())]
        return curr_q["class"]
    
    def predict(self, x):
        ret = []
        l = None
        for samp in x:
            l = pandas.DataFrame(numpy.array([samp]))
            if self.__rename__ is not None:   
                l = l.rename(columns=self.__rename__)
            ret.append(self.predict_sample(l))
        return ret
    

In [7]:
df = pandas.read_csv("./ApplesPears.csv")
df = df.rename(columns={"Class": "labels"})
df["Taste"] = df["Taste"].apply(lambda x: ["Sweet", "Sour", "Tart"].index(x))
dt = DecisionTree(df)
dt.test()

,predictions,Actual
0,Apple,Apple
1,Pear,Pear
2,Apple,Apple
3,Pear,Pear
4,Pear,Pear
5,Apple,Apple
6,Pear,Pear
7,Apple,Apple
8,Apple,Apple
9,Apple,Apple


In [208]:
class NaiveBayse(MachineLearning):
    
    def __init__(self, x, y=None, feat_thresh=4, tts=0.8, thresh=10):
        super(NaiveBayse, self).__init__(x, y, tts=0.8, thresh=10)
        self.probs = None
        self.__rename__ = None
        self.__feature_threshold__ = 4
        self.__df__ = pandas.DataFrame(self.xtrain)
        if type(x) == pandas.DataFrame:
            z = zip(list(df.columns.factorize()[0]),
                   list(df.columns.factorize()[1]))
            d = {key: value for (key, value) in z} 
            self.__rename__ = d
            self.__df__ = self.__df__.rename(columns=self.__rename__)
        else:
            raise RuntimeError("Please provide a Dataframe to Naive Bayse")
        self.__cols__ = self.__df__.columns
        self.__df__["labels"] = self.ytrain
        self.__classes__ = numpy.unique(self.__df__["labels"])
        self.fit()

    def gaus_pdf(self, x, mean, stdev):
        exponent = math.exp(-((x-mean)**2 / (2 * stdev**2 )))
        return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent

    def comp_prob(self, arr):
        mean = numpy.mean(arr)
        stdev = numpy.std(arr)
        return lambda x: self.gaus_pdf(x, mean, stdev)

    def simple_prob(self, arr):
        dummies = pandas.get_dummies(pandas.DataFrame(arr))
        p = {}
        for col in dummies.columns:
            p[col] = dummies[col].sum() / len(dummies)
        return p

    def f_prob(self, f, c):
        ret = None
        df_ = self.__df__[self.__df__["labels"] == c]
        if len(self.__df__[f].unique()) < self.__feature_threshold__:
            ret = self.simple_prob(df_[f])
        else:
            ret = self.comp_prob(df_[f])
        return ret

    def fit(self):
        probabilties = []
        for c in self.__classes__:
            p_class = (len(self.__df__[self.__df__["labels"] == c])/len(self.__df__))
            probs = {
                "class": c,
                "probability": p_class
            }
            for f in self.__cols__:
                probs[f] = self.f_prob(f, c)
            probabilties.append(probs)
        self.probs = probabilties
        
    def __samp_probabilities__(self, x):
        class_probs = []
        for c in probs:
            p = c["probability"]
            for f in list(c.keys())[2:]:
                if type(c[f]) is dict:
                    if (f+"_"+x["Greeness"].item()) in c[f]:
                        p *= c[f][f+"_"+x["Greeness"].item()]
                    else:
                        p *= 0.001
                else:
                    p *= c[f](x[f])
            class_probs.append(p)
        return class_probs
        
    def predict_sample(self, x):
        c_probs = self.__samp_probabilities__(x)
        norm_probs = [(i/(sum(class_probs)) ) for i in c_probs]
        class_ = self.probs[norm_probs.index(max(norm_probs))]["class"]
        return class_
    
    def predict(self, x):
        ret = []
        l = None
        for samp in x:
            l = pandas.DataFrame(numpy.array([samp]))
            if self.__rename__ is not None:   
                l = l.rename(columns=self.__rename__)
            ret.append(self.predict_sample(l))
        return ret


In [209]:
df = pandas.read_csv("./ApplesPears.csv")
df = df.rename(columns={"Class": "labels"})
df["Greeness"] = df["Greeness"].apply(lambda x: "pale" if x < 190 else ("medium" if x < 110 else "Dark"))
nb = NaiveBayse(df)
probs = nb.probs
nb.predict(nb.xtest)

['Apple',
 'Pear',
 'Apple',
 'Pear',
 'Pear',
 'Apple',
 'Pear',
 'Apple',
 'Apple',
 'Apple']